In [13]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import pandas as pd 
import math

In [14]:
def run(train_data, test_data, token):
#     train_temp = pd.DataFrame(data=train_data)
    train_diabete = train_data[train_data['label'] == 1]
    train_healthy = train_data[train_data['label'] == 0]
    
    # Get p_diabete and p_healthy
    label = np.array(train_data.label)
    label_bin = np.bincount(label)
    p_diabete = label_bin[1]/(label_bin[0] + label_bin[1])
    p_healthy = 1 - p_diabete
    
    # get mean and variance for healthy patients
    temp = train_healthy.loc[:,token]
    mean_healthy = np.nanmean(temp, axis=0)
    var_healthy = np.nanvar(temp, axis=0)
    
    # get mean and variance for diabete patients
    temp = train_diabete.loc[:,token]
    mean_diabete = np.nanmean(temp, axis=0)
    var_diabete = np.nanvar(temp, axis=0)
    
    # treating test data
    test_input  = test_data.loc[:,token]
    test_label  = test_data.loc[:,"label"].values
    
    diabete_rate = np.log(norm.pdf(test_input, mean_diabete, np.sqrt(var_diabete)))
    diabete_rate = [np.sum(arr) + np.log(p_diabete) for arr in diabete_rate]
    healthy_rate = np.log(norm.pdf(test_input, mean_healthy, np.sqrt(var_healthy)))
    healthy_rate = [np.sum(arr) + np.log(p_healthy) for arr in healthy_rate]
    
   # diabete_rate>healthy_rate, 152
    p = 0
    for i in range(152):
        if(test_label[i] == int(diabete_rate[i] >= healthy_rate[i])):
            p += 1
    return (p/152)


In [15]:
data = pd.read_csv("pima-indians-diabetes.csv") 
data.columns = ["preg", "plas", "pres", "skin", "test", "mass", "pedi", "age", "label"]

p1 = 0
for i in range(10):
    data = data.sample(frac=1)
    train_data = data[:615]
    test_data = data[615:]
    p1 += run(train_data, test_data, ["preg", "plas", "pres", "skin", "test", "mass", "pedi", "age"])

In [16]:
# data['test'][data['test'] == 0] = np.nan
data['pres'] = data['pres'].replace(0, np.nan)
data['skin'] = data['skin'].replace(0, np.nan)
data['mass'] = data['mass'].replace(0, np.nan)
data['age' ] = data['age' ].replace(0, np.nan)

In [17]:
p2 = 0
for i in range(10):
    data = data.sample(frac=1)
    train_data = data[:615]
    test_data = data[615:]
    p2 += run(train_data, test_data, ["preg", "plas", "pres", "skin", "test", "mass", "pedi", "age"])

In [19]:
print("Part One: ", p1/10, "\nPart Two: ", p2/10)

Part One:  0.7651315789473684 
Part Two:  0.7157894736842104
